In [1]:
# Import Dependencies for Machine Learning

import joblib
import pickle
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import sqlalchemy
import sqlite3 as sq
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

warnings.filterwarnings('ignore')

C:\Users\reeda\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\reeda\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\reeda\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\reeda\anaconda3\envs\mlenv\lib\site-packages\tensorflow\python\f

In [2]:
#import dataset
url = "https://drive.google.com/file/d/1uG3yPNRihgE3j2bGQGOAq8iNgMjXlybw/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
weather_df = pd.read_csv(path, index_col=0)

In [3]:
weather_df.head(10)

,City,State,Country,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow,Snow Depth,Wind Speed,Wind Direction,Wind Gust,Visibility,Cloud Cover,Relative Humidity,Conditions
0,Brentwood,TN,United States,2018-01-01,21.0,6.6,13.2,-7.7,NaN,0.0,0.0,0.0,13.7,291.50,18.3,9.9,27.4,58.76,Partially cloudy
1,Fayetteville,TN,United States,2018-01-01,22.0,10.8,14.7,-5.6,NaN,0.0,0.0,0.0,15.6,123.46,22.5,9.7,32.5,62.40,Partially cloudy
2,Franklin,TN,United States,2018-01-01,20.7,6.6,13.1,-7.4,NaN,0.0,0.0,0.0,13.8,306.21,18.3,9.9,26.5,59.63,Partially cloudy
3,Hendersonville,TN,United States,2018-01-01,21.1,6.8,13.3,-7.5,NaN,0.0,0.0,0.0,13.4,292.13,19.7,9.9,32.2,58.99,Partially cloudy
4,Nashville,TN,United States,2018-01-01,21.7,7.1,13.6,-7.9,NaN,0.0,0.0,0.0,13.9,278.04,21.9,9.9,32.1,57.18,Partially cloudy
5,Brentwood,TN,United States,2018-01-02,25.4,4.3,14.1,-6.4,NaN,0.0,0.0,0.0,7.4,165.92,NaN,9.9,16.4,52.74,Clear
6,Fayetteville,TN,United States,2018-01-02,28.7,5.7,16.2,-6.0,NaN,0.0,0.0,0.0,9.3,184.46,NaN,9.9,14.2,55.45,Clear
7,Franklin,TN,United States,2018-01-02,25.1,4.0,13.8,-6.4,NaN,0.0,0.0,0.0,7.3,193.88,NaN,9.9,12.3,53.90,Clear
8,Hendersonville,TN,United States,2018-01-02,25.0,4.3,13.9,-6.8,NaN,0.0,0.0,0.0,7.9,166.08,NaN,9.9,18.0,52.93,Clear
9,Nashville,TN,United States,2018-01-02,25.8,4.8,14.6,-6.9,NaN,0.0,0.0,0.0,8.8,153.88,NaN,9.9,24.8,50.71,Clear


In [4]:
#import aqi dataset
url = "https://drive.google.com/file/d/1pvPyRj--mbnipGaNnXDH_Fh7ncFvVZWO/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
air_df = pd.read_csv(path, index_col=0)

In [5]:
air_df.head(10)

,state,county,city,date_local,parameter,sample_duration,units_of_measure,arithmetic_mean,aqi
3911,Tennessee,Dyer,Dyersburg,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),3.2,13.0
39160,Tennessee,McMinn,Athens,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.9,25.0
22527,Tennessee,Hamilton,Chattanooga,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),7.6,32.0
12304,Tennessee,Montgomery,Clarksville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),7.2,30.0
28312,Tennessee,Madison,Jackson,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),4.9,20.0
36466,Tennessee,Loudon,Loudon,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),4.7,20.0
17530,Tennessee,Lawrence,Loretto,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.3,26.0
26553,Tennessee,Putnam,Cookeville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.8,28.0
13980,Tennessee,Sumner,Hendersonville,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),6.2,26.0
2127,Tennessee,Maury,Columbia,2018-01-01,PM2.5 - Local Conditions,24-HR BLK AVG,Micrograms/cubic meter (LC),5.9,25.0


In [6]:
weather_df.columns

Index(['City', 'State', 'Country', 'Date time', 'Maximum Temperature',
       'Minimum Temperature', 'Temperature', 'Wind Chill', 'Heat Index',
       'Precipitation', 'Snow', 'Snow Depth', 'Wind Speed', 'Wind Direction',
       'Wind Gust', 'Visibility', 'Cloud Cover', 'Relative Humidity',
       'Conditions'],
      dtype='object')

In [7]:
weather_df_clean = weather_df.drop(columns=["State", "Country",
                         "Maximum Temperature", "Minimum Temperature", "Wind Chill", "Snow Depth", "Wind Gust",
                         "Heat Index", "Wind Direction", "Snow", "Visibility", "Cloud Cover" ])
weather_df_clean.head(10)

,City,Date time,Temperature,Precipitation,Wind Speed,Relative Humidity,Conditions
0,Brentwood,2018-01-01,13.2,0.0,13.7,58.76,Partially cloudy
1,Fayetteville,2018-01-01,14.7,0.0,15.6,62.40,Partially cloudy
2,Franklin,2018-01-01,13.1,0.0,13.8,59.63,Partially cloudy
3,Hendersonville,2018-01-01,13.3,0.0,13.4,58.99,Partially cloudy
4,Nashville,2018-01-01,13.6,0.0,13.9,57.18,Partially cloudy
5,Brentwood,2018-01-02,14.1,0.0,7.4,52.74,Clear
6,Fayetteville,2018-01-02,16.2,0.0,9.3,55.45,Clear
7,Franklin,2018-01-02,13.8,0.0,7.3,53.90,Clear
8,Hendersonville,2018-01-02,13.9,0.0,7.9,52.93,Clear
9,Nashville,2018-01-02,14.6,0.0,8.8,50.71,Clear


In [8]:
weather_df_clean = weather_df_clean.rename(columns={"Date time": "date_time", "Wind Speed": "wind_speed",
                                                   "Relative Humidity": "relative_humidity"})
weather_df_clean.columns


Index(['City', 'date_time', 'Temperature', 'Precipitation', 'wind_speed',
       'relative_humidity', 'Conditions'],
      dtype='object')

In [9]:
air_df.columns

Index(['state', 'county', 'city', 'date_local', 'parameter', 'sample_duration',
       'units_of_measure', 'arithmetic_mean', 'aqi'],
      dtype='object')

In [10]:
air_df_clean  = air_df.drop(columns=['state', 'county','parameter', 'sample_duration','units_of_measure', 'arithmetic_mean',])
air_df_clean.head(10)

,city,date_local,aqi
3911,Dyersburg,2018-01-01,13.0
39160,Athens,2018-01-01,25.0
22527,Chattanooga,2018-01-01,32.0
12304,Clarksville,2018-01-01,30.0
28312,Jackson,2018-01-01,20.0
36466,Loudon,2018-01-01,20.0
17530,Loretto,2018-01-01,26.0
26553,Cookeville,2018-01-01,28.0
13980,Hendersonville,2018-01-01,26.0
2127,Columbia,2018-01-01,25.0


In [11]:
air_df_clean = air_df_clean.rename(columns={"date_local": "date_time"})
weather_df_clean.columns


Index(['City', 'date_time', 'Temperature', 'Precipitation', 'wind_speed',
       'relative_humidity', 'Conditions'],
      dtype='object')

In [12]:
sql_data = 'pollution.sqlite'


# Create connection & push the data:

conn = sq.connect(sql_data)
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS "WEATHER";
CREATE TABLE "WEATHER" (
	"index" INTEGER PRIMARY KEY AUTOINCREMENT,
	"city" TEXT NOT NULL,
	"date_time" TEXT NOT NULL,
	"Temperature" INTEGER NOT NULL,
	"Precipitation" TEXT NOT NULL,
	"wind_speed" TEXT NOT NULL,
    "relative_humidity" TEXT NOT NULL,
    "Conditions" TEXT NOT NULL
);
DROP TABLE IF EXISTS "POLLUTION";
CREATE TABLE "POLLUTION" (
	"index" INTEGER PRIMARY KEY AUTOINCREMENT,
	"city" TEXT NOT NULL,
	"date_time" INTEGER NOT NULL,
	"AQI" TEXT NOT NULL
);
''')
# conn.commit()
weather_df_clean.to_sql("WEATHER", conn, if_exists='append', index=True)
# conn.commit()
air_df_clean.to_sql("POLLUTION", conn, if_exists='append', index=True)

conn.commit()
conn.close()



In [13]:
# Reflect the Tables into SQLAlchemy ORM
engine = create_engine("sqlite:///pollution.sqlite")

# Reflect an existing database into a new model:
Base = automap_base()

# Reflect the tables:
Base.prepare(engine, reflect=True)


Base.classes.keys()

['POLLUTION', 'WEATHER']

In [14]:
pollution = Base.classes.POLLUTION
weather = Base.classes.WEATHER
con = sq.connect("pollution.sqlite")

combined_df = pd.read_sql_query("\
SELECT weather.city, weather.date_time, weather.Temperature,\
weather.Precipitation, weather.wind_speed, weather.relative_humidity, weather.Conditions,\
pollution.city, pollution.date_time, pollution.AQI FROM weather INNER JOIN POLLUTION ON weather.date_time = pollution.date_time WHERE weather.city = pollution.city", con)


combined_df

,city,date_time,Temperature,Precipitation,wind_speed,relative_humidity,Conditions,city,date_time,AQI
0,Hendersonville,2018-01-01,13.3,0.0,13.4,58.99,Partially cloudy,Hendersonville,2018-01-01,26.0
1,Nashville,2018-01-01,13.6,0.0,13.9,57.18,Partially cloudy,Nashville,2018-01-01,21.0
2,Hendersonville,2018-01-02,13.9,0.0,7.9,52.93,Clear,Hendersonville,2018-01-02,23.0
3,Nashville,2018-01-02,14.6,0.0,8.8,50.71,Clear,Nashville,2018-01-02,16.0
4,Hendersonville,2018-01-03,24.5,0.01,11.9,50.06,"Rain, Partially cloudy",Hendersonville,2018-01-03,22.0
...,...,...,...,...,...,...,...,...,...,...
1667,Nashville,2020-12-30,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",Nashville,2020-12-30,36.0
1668,Nashville,2020-12-30,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",Nashville,2020-12-30,39.0
1669,Hendersonville,2020-12-31,41.0,0.9,16.8,91.89,"Rain, Overcast",Hendersonville,2020-12-31,4.0
1670,Nashville,2020-12-31,41.6,0.86,17.4,90.33,"Rain, Overcast",Nashville,2020-12-31,13.0


In [15]:
combined_df2 = combined_df.drop(columns=["city", "date_time"])

combined_df2

,Temperature,Precipitation,wind_speed,relative_humidity,Conditions,AQI
0,13.3,0.0,13.4,58.99,Partially cloudy,26.0
1,13.6,0.0,13.9,57.18,Partially cloudy,21.0
2,13.9,0.0,7.9,52.93,Clear,23.0
3,14.6,0.0,8.8,50.71,Clear,16.0
4,24.5,0.01,11.9,50.06,"Rain, Partially cloudy",22.0
...,...,...,...,...,...,...
1667,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",36.0
1668,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",39.0
1669,41.0,0.9,16.8,91.89,"Rain, Overcast",4.0
1670,41.6,0.86,17.4,90.33,"Rain, Overcast",13.0


In [16]:
combined_df2["AQI"] = combined_df2.AQI.astype(float)
combined_df2["wind_speed"] = combined_df2.wind_speed.astype(float)
combined_df2["relative_humidity"] = combined_df2.relative_humidity.astype(float)
combined_df2["Precipitation"] = combined_df2.Precipitation.astype(float)
combined_df2.dtypes

Temperature          float64
Precipitation        float64
wind_speed           float64
relative_humidity    float64
Conditions            object
AQI                  float64
dtype: object

In [17]:
# Binning aqi values
aqi_bins = [-1, 50, 100, 300, 500]
group_names = ['Good', 'Moderate', 'Unhealthy', 'Hazardous']

# Categorize aqi based on the bins.
combined_df2["AQI_Range"] = pd.cut(combined_df2["AQI"], aqi_bins, labels=group_names)

combined_df2

,Temperature,Precipitation,wind_speed,relative_humidity,Conditions,AQI,AQI_Range
0,13.3,0.00,13.4,58.99,Partially cloudy,26.0,Good
1,13.6,0.00,13.9,57.18,Partially cloudy,21.0,Good
2,13.9,0.00,7.9,52.93,Clear,23.0,Good
3,14.6,0.00,8.8,50.71,Clear,16.0,Good
4,24.5,0.01,11.9,50.06,"Rain, Partially cloudy",22.0,Good
...,...,...,...,...,...,...,...
1667,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",36.0,Good
1668,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",39.0,Good
1669,41.0,0.90,16.8,91.89,"Rain, Overcast",4.0,Good
1670,41.6,0.86,17.4,90.33,"Rain, Overcast",13.0,Good


In [18]:
# Converting text to code value for AQI_Range
combined_df2["AQI_Range"] = combined_df2["AQI_Range"].astype('category').cat.codes
combined_df2 = combined_df2.drop(columns=["AQI"])
combined_df2

,Temperature,Precipitation,wind_speed,relative_humidity,Conditions,AQI_Range
0,13.3,0.00,13.4,58.99,Partially cloudy,0
1,13.6,0.00,13.9,57.18,Partially cloudy,0
2,13.9,0.00,7.9,52.93,Clear,0
3,14.6,0.00,8.8,50.71,Clear,0
4,24.5,0.01,11.9,50.06,"Rain, Partially cloudy",0
...,...,...,...,...,...,...
1667,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",0
1668,56.0,0.12,23.8,63.09,"Rain, Partially cloudy",0
1669,41.0,0.90,16.8,91.89,"Rain, Overcast",0
1670,41.6,0.86,17.4,90.33,"Rain, Overcast",0


In [19]:
application_cat = combined_df2.dtypes[combined_df2.dtypes == "object"].index.tolist()
application_cat

['Conditions']

In [20]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(combined_df2[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
# Merge one-hot encoded features and drop the original
combined_df2 = combined_df2.merge(encode_df, left_index=True,right_index=True)
combined_df2 = combined_df2.drop(columns = application_cat)
combined_df2.head()

,Temperature,Precipitation,wind_speed,relative_humidity,AQI_Range,Conditions_Clear,Conditions_Overcast,Conditions_Partially cloudy,Conditions_Rain,"Conditions_Rain, Overcast","Conditions_Rain, Partially cloudy",Conditions_Snow,"Conditions_Snow, Overcast","Conditions_Snow, Partially cloudy"
0,13.3,0.00,13.4,58.99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.6,0.00,13.9,57.18,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13.9,0.00,7.9,52.93,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14.6,0.00,8.8,50.71,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.5,0.01,11.9,50.06,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [22]:
# Split our preprocessed data into our features and target arrays
y = combined_df2["AQI_Range"].values
X = combined_df2.drop("AQI_Range", 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [23]:
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (1254, 13) (1254,)
Test (418, 13) (418,)


In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  75
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 75)                1050      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 6,151
Trainable params: 6,151
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
1254/1254 [==============================] - 0s 81us/sample - loss: 1.6729 - acc: 0.8931
Epoch 2/100
1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 3/100
1254/1254 [==============================] - 0s 22us/sample - loss: 1.6729 - acc: 0.8931
Epoch 4/100
1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 5/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 6/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 7/100
1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 8/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 9/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.89

1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 79/100
1254/1254 [==============================] - 0s 19us/sample - loss: 1.6729 - acc: 0.8931
Epoch 80/100
1254/1254 [==============================] - 0s 19us/sample - loss: 1.6729 - acc: 0.8931
Epoch 81/100
1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 82/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 83/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 84/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 85/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 0.8931
Epoch 86/100
1254/1254 [==============================] - 0s 21us/sample - loss: 1.6729 - acc: 0.8931
Epoch 87/100
1254/1254 [==============================] - 0s 20us/sample - loss: 1.6729 - acc: 

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

418/418 - 0s - loss: 1.2916 - acc: 0.9163
Loss: 1.2915626644517817, Accuracy: 0.9162679314613342


In [30]:
nn.save("weather_aqi.pkl")

In [34]:
load_model = tf.keras.models.load_model("weather_aqi.pkl")

In [35]:
load_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 75)                1050      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3800      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 6,151
Trainable params: 6,151
Non-trainable params: 0
_________________________________________________________________
